# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,f64,str,str,str,bool
"""75637911699970…","""../../../../da…",1.2927e19,"""13669925026803…","""47721421563518…","""17677253543433…",false
"""10706846025621…","""../../../../da…",1.4856e19,"""96868711387890…","""47721421563518…","""14629200209930…",true
"""17013778720464…","""../../../../da…",5.6458e17,"""10478570846839…","""10876764037856…","""17677253543433…",false
"""32495227445521…","""../../../../da…",4.0723e18,"""96868711387890…","""86065680383664…","""17677253543433…",false
"""55513510678997…","""../../../../da…",1.4677e19,"""13669925026803…","""86065680383664…","""64341157878450…",true
"""13787012272845…","""../../../../da…",1.6330e19,"""10478570846839…","""69897137468458…",null,false
"""79849999649325…","""../../../../da…",8.7675e18,"""10478570846839…","""86065680383664…","""64341157878450…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_City': 'PREDICTOR_0',
 'filename': 'filename',
 'Customer_CLV': 'PREDICTOR_2',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""14465027989772…","""../../../../da…",1.2927e19,"""12837118204749…","""FirstMortgage3…","""42742600268414…","""Rejected"""
"""15864797487647…","""../../../../da…",1.4856e19,"""13038335467387…","""FirstMortgage3…","""16968007675717…","""Accepted"""
"""11312459237682…","""../../../../da…",5.6458e17,"""12899434046411…","""MoneyMarketSav…","""42742600268414…","""Rejected"""
"""11254416262543…","""../../../../da…",4.0723e18,"""13038335467387…","""BasicChecking""","""42742600268414…","""Rejected"""
"""68822884280512…","""../../../../da…",1.4677e19,"""12837118204749…","""BasicChecking""","""82751290886829…","""Accepted"""
"""13532627541031…","""../../../../da…",1.6330e19,"""12899434046411…","""UPlusFinPerson…",null,"""Rejected"""
"""47735001256021…","""../../../../da…",8.7675e18,"""12899434046411…","""BasicChecking""","""82751290886829…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""10956173841986…","""../../../../da…",1.2927e19,"""14220657486805…","""FirstMortgage3…","""17952503333097…","""Rejected"""
"""16574171319331…","""../../../../da…",1.4856e19,"""16160299586719…","""FirstMortgage3…","""13771885637361…","""Accepted"""
"""36510205268633…","""../../../../da…",5.6458e17,"""13702922554313…","""MoneyMarketSav…","""17952503333097…","""Rejected"""
"""70546946133517…","""../../../../da…",4.0723e18,"""16160299586719…","""BasicChecking""","""17952503333097…","""Rejected"""
"""17708367332691…","""../../../../da…",1.4677e19,"""14220657486805…","""BasicChecking""","""35787173284252…","""Accepted"""
"""36514085505050…","""../../../../da…",1.6330e19,"""13702922554313…","""UPlusFinPerson…",null,"""Rejected"""
"""16175581296741…","""../../../../da…",8.7675e18,"""13702922554313…","""BasicChecking""","""35787173284252…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_City=PREDICTOR_0
filename=filename
Customer_CLV=PREDICTOR_2
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""18189975885507…",1.2927e19,"""16843731726272…","""FirstMortgage3…","""10900894234423…","""Rejected"""
"""14040108070101…",1.4856e19,"""11140134536949…","""FirstMortgage3…","""17919745516940…","""Accepted"""
"""13976561677151…",5.6458e17,"""13780435374893…","""MoneyMarketSav…","""10900894234423…","""Rejected"""
"""49567413994904…",4.0723e18,"""11140134536949…","""BasicChecking""","""10900894234423…","""Rejected"""
"""42269147014725…",1.4677e19,"""16843731726272…","""BasicChecking""","""59533486519071…","""Accepted"""
"""15588292508397…",1.6330e19,"""13780435374893…","""UPlusFinPerson…",null,"""Rejected"""
"""63811838581705…",8.7675e18,"""13780435374893…","""BasicChecking""","""59533486519071…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'